In [2]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from src.filter import GaussianOutlierFilter
from src.extractor import LeidenRepresentativeSelector

import pandas as pd

import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv("/workspace/01_data-extractor/data/TG_GATE_tables/merged_animal_data_final.csv", encoding='shift-jis')

In [6]:
data.iloc[:, 13:56]

,TERMINAL_BW(g),LIVER(g),KIDNEY_TOTAL(g),KIDNEY_R(g),KIDNEY_L(g),ALP(IU/L),TC(mg/dL),TG(mg/dL),PL(mg/dL),TBIL(mg/dL),...,Plat(x10_4/uL),WBC(x10_2/uL),Neu(%),Eos(%),Bas(%),Mono(%),Lym(%),PT(s),APTT(s),Fbg(mg/dL)
0,209.5,10.548,1.886,0.940,0.946,877.0,104.0,83.0,172.0,0.29,...,143.6,16.4,17.0,3.0,1.0,16.0,63.0,12.1,13.7,207.0
1,183.3,8.811,1.845,0.928,0.917,917.0,79.0,30.0,137.0,0.16,...,137.5,17.8,15.0,2.0,0.0,3.0,81.0,11.6,15.1,182.0
2,200.8,9.760,1.956,0.991,0.965,751.0,86.0,41.0,150.0,0.19,...,117.5,43.5,25.0,2.0,0.0,11.0,61.0,12.3,16.5,189.0
3,210.9,9.525,1.949,0.968,0.981,742.0,79.0,28.0,136.0,0.33,...,106.4,27.3,14.0,6.0,0.0,26.0,54.0,12.2,16.8,168.0
4,196.3,9.170,1.961,0.947,1.014,923.0,79.0,27.0,128.0,0.26,...,110.1,12.2,9.0,14.0,0.0,21.0,57.0,13.6,17.7,158.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23863,217.3,8.790,1.800,0.900,0.900,1131.0,102.0,41.1,150.0,0.11,...,120.7,105.1,13.0,1.0,0.0,3.0,82.0,NaN,NaN,NaN
23864,214.2,8.930,1.890,0.950,0.940,971.0,134.0,46.7,172.0,0.14,...,117.4,66.2,28.0,1.0,0.0,4.0,68.0,NaN,NaN,NaN
23865,196.3,8.860,1.900,1.000,0.900,992.0,113.0,25.5,144.0,0.06,...,143.9,72.8,42.0,0.0,0.0,3.0,54.0,NaN,NaN,NaN
23866,184.4,7.150,1.710,0.850,0.860,1152.0,96.0,44.9,113.0,0.12,...,123.9,83.1,60.0,0.0,0.0,4.0,35.0,NaN,NaN,NaN


In [7]:
gauss_filter = GaussianOutlierFilter(1,)
filtered_data = gauss_filter.fit_transform(data, ["ALP(IU/L)", "ALT(IU/L)"])

Alert! ALP(IU/L) の正規性が否定されました (p=0.0010)。IQR法に切り替えます。
Alert! ALT(IU/L) の正規性が否定されました (p=0.0010)。IQR法に切り替えます。
Filtering completed 23868 -> 2106


In [18]:
selector = LeidenRepresentativeSelector(n_neighbors=1, resolution=3000)
_, clustered_indices = selector.fit_transform(data, start_index=13, end_index=56, pca_threshold=0.9)

Clustering completed: 8353 clusters found, 8353 representative points selected.


In [19]:
data.iloc[clustered_indices]

,EXP_ID,GROUP_ID,INDIVIDUAL_ID,COMPOUND_NAME,SINGLE_REPEAT_TYPE,SEX_TYPE,STRAIN_TYPE,ADMINISTRATION_ROUTE_TYPE,ANIMAL_AGE(week),SACRIFICE_PERIOD,...,Neu(%),Eos(%),Bas(%),Mono(%),Lym(%),PT(s),APTT(s),Fbg(mg/dL),IMAGE_ID_Kidney,IMAGE_ID_Liver
1497,113,12,3,indomethacin,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,29 day,...,9.0,1.0,0.0,4.0,86.0,12.1,17.8,250.0,27503.0,27395.0
9184,325,13,5,nitrofurazone,Single,Male,Crj:CD(SD)IGS,Gavage,6,3 hr,...,19.0,1.0,0.0,5.0,74.0,15.0,16.4,230.0,16175.0,16173.0
14522,465,15,5,mexiletine,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,15 day,...,19.0,1.0,0.0,2.0,78.0,16.1,19.9,305.0,53905.0,53904.0
15712,510,1,5,acarbose,Single,Male,Crj:CD(SD)IGS,Gavage,6,3 hr,...,22.0,1.0,0.0,3.0,72.0,13.7,17.4,283.0,41535.0,41390.0
22789,688,1,2,butylated hydroxyanisole,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,4 day,...,22.2,0.8,0.2,1.9,74.4,11.2,12.5,257.0,64319.0,64318.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23706,740,16,4,bortezomib,Single,Male,Crj:CD(SD)IGS,intravenous injection,6,24 hr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23803,745,4,1,gefitinib,Single,Male,Crj:CD(SD)IGS,Gavage,6,24 hr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66878.0,66875.0
1420,112,13,1,indomethacin,Single,Male,Crj:CD(SD)IGS,Gavage,6,3 hr,...,8.0,0.0,0.0,9.0,83.0,12.8,15.5,223.0,5443.0,NaN
1126,101,2,2,naphthyl isothiocyanate,Repeat,Male,Crj:CD(SD)IGS,Gavage,6,8 day,...,10.0,0.0,0.0,2.0,87.0,12.1,20.8,224.0,25476.0,25283.0


In [10]:
import torch
# 2次元テンソル
data_2d = torch.tensor([[[1.0], [2.0]], [[3.0], [4.0]], [[5.0], [6.0]]])

# 列ごとの平均を計算 (dim=0)
mean_value_dim0 = torch.mean(data_2d, dim=0, keepdim=True)
print(mean_value_dim0) 
# 行ごとの平均を計算 (dim=1)
mean_value_dim1 = torch.mean(data_2d, dim=1)
print(mean_value_dim1)


tensor([[[3.],
         [4.]]])
tensor([[1.5000],
        [3.5000],
        [5.5000]])


In [ ]:

data_2d.shape

torch.Size([3, 2, 1])

In [5]:
data_2d - mean_value_dim0

tensor([[-2., -2.],
        [ 0.,  0.],
        [ 2.,  2.]])